In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Train.csv')

In [3]:
df.isnull().sum()[df.isnull().sum()!=0]

ecec20         5
hp20           5
xhp20          5
BulkDensity    4
dtype: int64

In [4]:
for column in df.columns:
  if df[column].isnull().any():
    df[column].fillna(df[column].mean(), inplace=True)
    

In [5]:
# Load datasets
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')
train_gap_df = pd.read_csv('Gap_Train.csv')
test_gap_df = pd.read_csv('Gap_Test.csv')
sample_submission = pd.read_csv('SampleSubmission.csv')

In [6]:
df.drop(columns=['site','lon','lat'],inplace=True)
test_df.drop(columns=['site','lon','lat'],inplace=True)

In [7]:
modis_1=pd.read_csv('modis_1_clean.csv')
modis_2=pd.read_csv('modis_2_clean.csv')
modis_3=pd.read_csv('modis_3_clean.csv')
modis_4=pd.read_csv('modis_4_clean.csv')
modis_5=pd.read_csv('modis_5_clean.csv')
sentinel_1=pd.read_csv('sentinel_1_clean.csv')
sentinel_2=pd.read_csv('sentinel_2_clean.csv')
landsat=pd.read_csv('landsat_clean.csv')

In [8]:
landsat['date'] = pd.to_datetime(landsat['date'])
modis_1['date'] = pd.to_datetime(modis_1['date'])
modis_2['date'] = pd.to_datetime(modis_2['date'])
modis_3['date'] = pd.to_datetime(modis_3['date'])
modis_4['date'] = pd.to_datetime(modis_4['date'])
modis_5['date'] = pd.to_datetime(modis_5['date'])
sentinel_1['date'] = pd.to_datetime(sentinel_1['date'])
sentinel_2['date'] = pd.to_datetime(sentinel_2['date'])

In [9]:
satelite_df=pd.DataFrame()
satelite_test_df=pd.DataFrame()

In [10]:
satelite_df['PID']=df['PID']
satelite_test_df['PID']=test_df['PID']

In [11]:
date_range = pd.date_range(start='2017-01-01', end='2019-12-31', freq='D')
def expand_with_dates(df, date_range):
    return (
        df[['PID']]
        .drop_duplicates()
        .assign(key=1)
        .merge(pd.DataFrame({'date': date_range, 'key': 1}), on='key')
        .drop(columns='key')
        .sort_values(['PID', 'date'])
        .reset_index(drop=True)
    )
satelite_df = expand_with_dates(satelite_df, date_range)
satelite_test_df = expand_with_dates(satelite_test_df, date_range)


# merging

### modis_5

In [12]:
satelite_df = satelite_df.merge(
    modis_5, on=['PID', 'date'], how='left'
)

In [13]:
satelite_test_df = satelite_test_df.merge(
    modis_5, on=['PID', 'date'], how='left'
)

### landsat

In [14]:

satelite_df_indexed = satelite_df.set_index(['PID', 'date'])
landsat_indexed = landsat.set_index(['PID', 'date'])
landsat_filtered = landsat_indexed.loc[
    satelite_df_indexed.index.intersection(landsat_indexed.index)
]
satelite_df_combined = landsat_filtered.combine_first(satelite_df_indexed)


satelite_df = satelite_df_combined.reset_index()


In [15]:

satelite_test_df_indexed = satelite_test_df.set_index(['PID', 'date'])
landsat_indexed = landsat.set_index(['PID', 'date'])


landsat_filtered = landsat_indexed.loc[
    satelite_test_df_indexed.index.intersection(landsat_indexed.index)
]

satelite_test_df_combined = landsat_filtered.combine_first(satelite_test_df_indexed)


satelite_test_df = satelite_test_df_combined.reset_index()


### modis_4

In [16]:
modis_4.columns

Index(['PID', 'date', 'b01', 'b02', 'b03', 'b07', 'EVI', 'NDVI'], dtype='object')

In [17]:
# 1️⃣ Index both on PID + date
satelite_df_indexed = satelite_df.set_index(['PID', 'date'])
modis_4_indexed = modis_4.set_index(['PID', 'date'])

# 2️⃣ Filter modis_4 rows that match satelite_df
modis_4_filtered = modis_4_indexed.loc[
    satelite_df_indexed.index.intersection(modis_4_indexed.index)
]

# 3️⃣ Combine: fill NaNs in satelite_df with modis_4 and add any new columns (EVI, NDVI)
satelite_df_combined = modis_4_filtered.combine_first(satelite_df_indexed)

# 4️⃣ Reset index back to columns
satelite_df = satelite_df_combined.reset_index()


In [18]:
# 1️⃣ Index both on PID + date
satelite_test_df_indexed = satelite_test_df.set_index(['PID', 'date'])
modis_4_indexed = modis_4.set_index(['PID', 'date'])

# 2️⃣ Filter modis_4 rows that match satelite_test_df
modis_4_filtered = modis_4_indexed.loc[
    satelite_test_df_indexed.index.intersection(modis_4_indexed.index)
]

# 3️⃣ Combine: fill NaNs in satelite_test_df with modis_4 and add any new columns (EVI, NDVI)
satelite_test_df_combined = modis_4_filtered.combine_first(satelite_test_df_indexed)

# 4️⃣ Reset index back to columns
satelite_test_df = satelite_test_df_combined.reset_index()


In [19]:
satelite_df.isnull().sum()

PID             0
date            0
EVI       8063253
NDVI      8063253
ST_B10    8111163
b01       2281284
b02       2281284
b03       2281284
b04       2319142
b05       8111163
b06       8111163
b07       7741346
dtype: int64

In [20]:
satelite_test_df.isnull().sum()

PID             0
date            0
EVI       2518574
NDVI      2518574
ST_B10    2561728
b01        912916
b02        912916
b03        912916
b04        937214
b05       2561728
b06       2561728
b07       2438381
dtype: int64

### modis_1

In [21]:
# 1️⃣ Index both on PID + date
satelite_df_indexed = satelite_df.set_index(['PID', 'date'])
modis_1_indexed = modis_1.set_index(['PID', 'date'])

# 2️⃣ Filter modis_1 rows that match satelite_df
modis_1_filtered = modis_1_indexed.loc[
    satelite_df_indexed.index.intersection(modis_1_indexed.index)
]

# 3️⃣ Combine: fill NaNs in satelite_df with modis_1 and add any new columns (EVI, NDVI)
satelite_df_combined = modis_1_filtered.combine_first(satelite_df_indexed)

# 4️⃣ Reset index back to columns
satelite_df = satelite_df_combined.reset_index()


In [22]:
# 1️⃣ Index both on PID + date
satelite_test_df_indexed = satelite_test_df.set_index(['PID', 'date'])
modis_1_indexed = modis_1.set_index(['PID', 'date'])

# 2️⃣ Filter modis_1 rows that match satelite_test_df
modis_1_filtered = modis_1_indexed.loc[
    satelite_test_df_indexed.index.intersection(modis_1_indexed.index)
]

# 3️⃣ Combine: fill NaNs in satelite_test_df with modis_1 and add any new columns (EVI, NDVI)
satelite_test_df_combined = modis_1_filtered.combine_first(satelite_test_df_indexed)

# 4️⃣ Reset index back to columns
satelite_test_df = satelite_test_df_combined.reset_index()


In [23]:
satelite_df.isnull().sum()

PID             0
date            0
EVI       8069295
NDVI      8069295
ST_B10    8117205
b01       1804093
b02       1804093
b03       1804093
b04       1817448
b05       2395625
b06       2395625
b07       2307544
dtype: int64

### modis_2

In [24]:
modis_2.columns

Index(['PID', 'date', 'LST_Day_1km', 'LST_Night_1km'], dtype='object')

In [25]:
# 1️⃣ Index both on PID + date
satelite_df_indexed = satelite_df.set_index(['PID', 'date'])
modis_2_indexed = modis_2.set_index(['PID', 'date'])

# 2️⃣ Filter modis_2 to only rows matching satelite_df index
modis_2_filtered = modis_2_indexed.loc[
    satelite_df_indexed.index.intersection(modis_2_indexed.index)
]

# 3️⃣ Combine: fill NaNs, add new columns
satelite_df_indexed = modis_2_filtered.combine_first(satelite_df_indexed)

# 4️⃣ Reset index to get PID/date back as columns
satelite_df = satelite_df_indexed.reset_index()


In [26]:
# 1️⃣ Index both on PID + date
satelite_test_df_indexed = satelite_test_df.set_index(['PID', 'date'])
modis_2_indexed = modis_2.set_index(['PID', 'date'])

# 2️⃣ Filter modis_2 to only rows matching satelite_test_df index
modis_2_filtered = modis_2_indexed.loc[
    satelite_test_df_indexed.index.intersection(modis_2_indexed.index)
]

# 3️⃣ Combine: fill NaNs, add new columns
satelite_test_df_indexed = modis_2_filtered.combine_first(satelite_test_df_indexed)

# 4️⃣ Reset index to get PID/date back as columns
satelite_test_df = satelite_test_df_indexed.reset_index()



In [27]:
satelite_df.isnull().sum()

PID                    0
date                   0
EVI              8069295
LST_Day_1km      6497124
LST_Night_1km    6497124
NDVI             8069295
ST_B10           8117205
b01              1804093
b02              1804093
b03              1804093
b04              1817448
b05              2395625
b06              2395625
b07              2307544
dtype: int64

### modis_3

In [28]:
modis_3.columns

Index(['PID', 'date', 'ET', 'PET'], dtype='object')

In [29]:
# 1️⃣ Index both on PID + date
satelite_df_indexed = satelite_df.set_index(['PID', 'date'])
modis_3_indexed = modis_3.set_index(['PID', 'date'])

# 2️⃣ Filter modis_3 to only rows matching satelite_df index
modis_3_filtered = modis_3_indexed.loc[
    satelite_df_indexed.index.intersection(modis_3_indexed.index)
]

# 3️⃣ Combine: fill NaNs, add new columns
satelite_df_indexed = modis_3_filtered.combine_first(satelite_df_indexed)

# 4️⃣ Reset index to get PID/date back as columns
satelite_df = satelite_df_indexed.reset_index()


In [30]:
# 1️⃣ Index both on PID + date
satelite_test_df_indexed = satelite_test_df.set_index(['PID', 'date'])
modis_3_indexed = modis_3.set_index(['PID', 'date'])

# 2️⃣ Filter modis_3 to only rows matching satelite_test_df index
modis_3_filtered = modis_3_indexed.loc[
    satelite_test_df_indexed.index.intersection(modis_3_indexed.index)
]

# 3️⃣ Combine: fill NaNs, add new columns
satelite_test_df_indexed = modis_3_filtered.combine_first(satelite_test_df_indexed)

# 4️⃣ Reset index to get PID/date back as columns
satelite_test_df = satelite_test_df_indexed.reset_index()



### sentinel_1

In [31]:
# 1️⃣ Index both on PID + date
satelite_df_indexed = satelite_df.set_index(['PID', 'date'])
sentinel_1_indexed = sentinel_1.set_index(['PID', 'date'])

# 2️⃣ Filter sentinel_1 to only rows matching satelite_df index
sentinel_1_filtered = sentinel_1_indexed.loc[
    satelite_df_indexed.index.intersection(sentinel_1_indexed.index)
]

# 3️⃣ Combine: fill NaNs, add new columns
satelite_df_indexed = sentinel_1_filtered.combine_first(satelite_df_indexed)

# 4️⃣ Reset index to get PID/date back as columns
satelite_df = satelite_df_indexed.reset_index()


In [32]:
# 1️⃣ Index both on PID + date
satelite_test_df_indexed = satelite_test_df.set_index(['PID', 'date'])
sentinel_1_indexed = sentinel_1.set_index(['PID', 'date'])

# 2️⃣ Filter sentinel_1 to only rows matching satelite_test_df index
sentinel_1_filtered = sentinel_1_indexed.loc[
    satelite_test_df_indexed.index.intersection(sentinel_1_indexed.index)
]

# 3️⃣ Combine: fill NaNs, add new columns
satelite_test_df_indexed = sentinel_1_filtered.combine_first(satelite_test_df_indexed)

# 4️⃣ Reset index to get PID/date back as columns
satelite_test_df = satelite_test_df_indexed.reset_index()



### sentinel_2

In [33]:
sentinel_2.columns

Index(['PID', 'date', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8',
       'B8A'],
      dtype='object')

In [34]:
# 1️⃣ Index both on PID + date
satelite_df_indexed = satelite_df.set_index(['PID', 'date'])
sentinel_2_indexed = sentinel_2.set_index(['PID', 'date'])

# 2️⃣ Filter sentinel_2 rows that match satelite_df
sentinel_2_filtered = sentinel_2_indexed.loc[
    satelite_df_indexed.index.intersection(sentinel_2_indexed.index)
]

# 3️⃣ Combine: fill NaNs in satelite_df with sentinel_2 and add any new columns (EVI, NDVI)
satelite_df_combined = sentinel_2_filtered.combine_first(satelite_df_indexed)

# 4️⃣ Reset index back to columns
satelite_df = satelite_df_combined.reset_index()


In [35]:
# 1️⃣ Index both on PID + date
satelite_test_df_indexed = satelite_test_df.set_index(['PID', 'date'])
sentinel_2_indexed = sentinel_2.set_index(['PID', 'date'])

# 2️⃣ Filter sentinel_2 rows that match satelite_test_df
sentinel_2_filtered = sentinel_2_indexed.loc[
    satelite_test_df_indexed.index.intersection(sentinel_2_indexed.index)
]

# 3️⃣ Combine: fill NaNs in satelite_test_df with sentinel_2 and add any new columns (EVI, NDVI)
satelite_test_df_combined = sentinel_2_filtered.combine_first(satelite_test_df_indexed)

# 4️⃣ Reset index back to columns
satelite_test_df = satelite_test_df_combined.reset_index()


In [37]:
satelite_df.to_csv('satelite_df.csv', index=False)
satelite_test_df.to_csv('satelite_test_df.csv', index=False)